In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import datetime as dt
import sys
import scipy.stats as stats
import urllib
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import torch
from torch import cuda




In [ ]:
test = pd.read_csv("C:/vscode/open/test.csv")
train = pd.read_csv("C:/vscode/open/train.csv")

In [6]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))


해당 프로젝트는 NULL값이 내재된 데이터에서 결과를 예측하는 반지도학습 프로젝트였습니다.

따라서 저도 결측값에 적절한 값을 배정하는 데에 가장 크게 중점을 두고 진행하였습니다.


# 칼럼 관리

In [ ]:
columns = train.columns

for i in columns:
    print("unique {}:{}".format(i, len(train[i].unique())))
    print("count null {}:{}".format(i, train[i].isnull().sum()))
    print("")

ID 칼럼은 분석에 직접적으로 이용되는 칼럼이 아니므로 별도로 보관합니다.

TABLE의 칼럼들 중, CANCELLED(항공편 취소됨)과 DIVERTED(경유여부) 칼럼은 NULL도 없으며 모든 관측값들이 동일합니다. 따라서 분석에 영향을 주지 못한다고 판단하여 해당 칼럼을 제거하였습니다.

또한, 컬럼들 중 Origin_Airport(출발 공항)와 Origin_Airport_ID(출발 공항 ID), Destination_Airport(도착 공항)와 Destination_Airport_ID(도착 공항 ID)는 각기 같은 값을 지칭합니다. 더욱이, Airline(항공사), Carrier_Code(IATA)(항공사 코드), Carrier_Code(DOT)값 역시 모두 같은 값들을 지칭합니다.

이에, 역시 해당 칼럼들도 Origin_Airport, Destination_Airport, Airline만을 빼고 모두 제거합니다. 다만, Carrier와 관련된 종류들은 이후 Airline의 결측을 메우는 데 사용할 수 있기 때문에 남겨둡니다.

따라서, 분석은 날짜(월+일), 시간(하루중 시간), 공항(출발-도착), 주(출발-도착), 항공사, 기종, 거리의 총 7가지 변수를 중심으로 이루어집니다. 

In [ ]:
ID = train['ID']

train.drop(columns = ['ID', 'Origin_Airport_ID', 'Destination_Airport_ID', 'Cancelled', 'Diverted'], inplace=True)

In [ ]:
train.columns

이들 중, NULL이 다수 포함된 칼럼은 Estimated_Departure_Time, 'Estimated_Arrival_Time', 'Origin_State', 'Destination_State', 'Airline'의 다섯 칼럼입니다. 

# Time칼럼 분단위 변환

테이블 내의 칼럼들 중, Estimated_Departure_Time과 Estimated_Arrival_Time은 각 출발시간과 도착시간입니다. 이들 칼럼은 HH:MM의 꼴로 되어 있습니다. 예를 들어, 6시 00분이라면 600, 저녁 6시 20분이라면 1820입니다. 
이를 바로 시간으로 바꾸어도 되지만, 이후 null값에 대한 계산상 편의를 위해 일단 분단위 시간으로 바꾸어주었습니다(그냥 제가 이게 더 직관적이어서 편하더라구요). 각 변환 시간의 배정 칼럼은 edt, eat로 설정하였습니다.

In [ ]:
i = 0
while i < len(train['Estimated_Departure_Time']):
    if train.loc[i, 'Estimated_Departure_Time'] > 0:
        train.loc[i, 'edt'] = (train.loc[i, 'Estimated_Departure_Time']//100)*60 + train.loc[i, 'Estimated_Departure_Time']%100
    i += 1

In [ ]:
i = 0
while i < len(train['Estimated_Arrival_Time']):
    if train.loc[i, 'Estimated_Arrival_Time'] > 0:
        train.loc[i, 'eat'] = (train.loc[i, 'Estimated_Arrival_Time']//100)*60 + train.loc[i, 'Estimated_Arrival_Time']%100
    i += 1

이후, 빈칸을 채우는 데 활용할 수 있도록, eat칼럼에 대한 변형 작업을 거칩니다.
eat < edt라면, 시간대가 변하지 않았다는 가정 하에서는 24시를 넘겨 다음날 도착했다고 보는 것이 타당하므로 해당 조건의 값들에 1440(하루를 분으로 표현)을 더해줍니다.

In [ ]:
i = 0
while i <= 999999:
    if train.loc[i, 'eat'] - train.loc[i, 'edt'] < 0:
        train.loc[i, 'eat'] += 1440
    i += 1

# 날짜(월, 일)칼럼 D(eparture)와 A로 별도 분리

추가로, 세분화해서 활용 가능하도록, Month와 Day_of_Month를 도착일자와 출발일자로 분리합니다. 출발 일자는 변경이 없겠지만, 도착일자는 위의 조건과 같은 상황에 따라 변동이 가능합니다.

In [ ]:
train['dmonth'] = train['Month']
train['dday'] = train['Day_of_Month']

In [ ]:
train['amonth'] = train['Month']
train['aday'] = train['Day_of_Month']

# Airport 칼럼을 통해서 States 칼럼 결측값 메우기

결측값이 있는 데이터는 전체 데이터중 주, 시간, 항공사이며, 결측값이 없는 데이터는 거리, 공항, 기종, 날짜입니다. 해당 데이터들 중, 주 데이터의 경우는 공항 데이터 값이 Unique 성질을 가지므로(제가 알기론 그렇습니다. 잘은 모릅니다), 공항 데이터를 통하여 주 데이터의 결측값을 메울 수 있으리라 예상하였습니다.

이에, 주별로 공항 데이터를 groupby하여 주 결측값이 있는 데이터에 공항을 통해 주값을 메워주도록 하였습니다. OS(Origin_State)와 DS(Destination_State)를 통해 값을 배정하였습니다.

In [ ]:
OS = train.groupby('Origin_State')['Origin_Airport'].unique().reset_index()
DS = train.groupby("Destination_State")['Destination_Airport'].unique().reset_index()

In [ ]:
type(OS['Origin_Airport'])

In [ ]:
i = 0
while i <= 999999:
    j = 0
    if isinstance(train.loc[i, 'Origin_State'], float):
        while j < 52:
            if train.loc[i, 'Origin_Airport'] in list(OS.loc[j, 'Origin_Airport']):
                train.loc[i, 'Origin_State'] = OS.loc[j, 'Origin_State']
            j += 1
    i += 1

In [ ]:
i = 0
while i <= 999999:
    j = 0
    if isinstance(train.loc[i, 'Destination_State'], float):
        while j < 52:
            if train.loc[i, 'Destination_Airport'] in list(DS.loc[j, 'Destination_Airport']):
                train.loc[i, 'Destination_State'] = DS.loc[j, 'Destination_State']
            j += 1
    i += 1

이렇게 하면 아무래도 100만개짜리 파일이니 어딘가엔 반드시 매칭되는 공항 정보와 주 정보가 있을 것이고, null값을 처리할 수 있을 거라 봅니다. 그러면 이제 train의 null 정보를 확인해보도록 하겠습니다.

In [ ]:
train[['Origin_State', 'Destination_State']].isnull().sum()

?

혹시 OS에만 있는 공항 정보인가 하여 Destination_State의 null 정보를 origin 정보를 통해 처리해보겠습니다.
생각해보니 진작에 통합할걸 그랬네요

In [ ]:
i = 0
while i <= 999999:
    j = 0
    if isinstance(train.loc[i, 'Destination_State'], float):
        while j < 52:
            if train.loc[i, 'Destination_Airport'] in list(OS.loc[j, 'Origin_Airport']):
                train.loc[i, 'Destination_State'] = DS.loc[j, 'Origin_State']
            j += 1
    i += 1

이제는 해결 됐겠죠. 한번 보죠.

In [ ]:
train[['Origin_State', 'Destination_State']].isnull().sum()

?

한번 뭐가 문제인지 찾아보도록 하겠습니다.

In [ ]:
train[train['Destination_State'].isnull()]

YNG라는 공항이네요. 묘하게 YG ent.가 생각나는 네이밍인데 저만 그런가요? 아무튼 해당 공항을 한번 전체 데이터에서 찾아보겠습니다.

In [ ]:
train[train['Origin_Airport'] == 'YNG']

In [ ]:
train[train['Destination_Airport'] == 'YNG']

예 이것밖에 없네요.

무슨 공항인가 해서 구글신에게 여쭤보니 Youngstown-Warren Regional Airport라고 하는 곳이라고 합니다. 진짜 살면서 처음 들어본 공항인데, 오하이오주라는 곳에 있다고 하네요. 저는 오하이오라는 주를 제가 직접 타이핑해본게 처음인거같은데 아무튼 그렇습니다. 더군다나 지역 공항이라 더 항공편수가 작은것같기도 합니다.

Dacon 룰 내에서 외부 정보를 활용하는건 규칙 위반이니 이후에 가능하면 항공사정보를 추출해서 가능한 값으로, 불가능하면 그냥 주별 최빈값으로 채워주도록 하겠습니다. 아마 후자라면 뉴욕 아니면 캘리포니아일것같네요.

# time 칼럼 결측

하여튼 주는 채웠고, 남은 값들은 항공사, 시간(time) 값이 되겠네요. 
우선 시간 값을 먼저 채워 보려고 합니다. 아까 Estimated_Departure|Arrival_Time 값을 분단위로 변환해서 edt, eat 값으로 변환했었는데, 이 값을 토대로 State별로의 이동 시간을 추적해서 빈 값을 채워넣으려고 합니다. 거리 정보는 모든 값이 다 있으니 그냥 사용할까도 생각했지만, 아무래도 꽤 큰 값이 있는거같던데, 잘못 들어간 값이 있을 수 있으니 한번 확인을 해 주고 가 볼까 합니다.

이후에는 여기에 더해서, Distance와 평균 시간당 이동을 통해서 별도의 소요시간 컬럼을 작성하려고 합니다. 아무래도 시차가 낀 데이터라 갑갑하네요.

우선은 Distance부터 한번 정리하겠습니다.

In [ ]:
train.describe()

In [ ]:
plt.hist(train['Distance'], bins=50)

보면 Distance의 대부분의 값들은 1100 안쪽으로, 일부 값들은 3천이 약간 안되는 값들이 있고, 이후 아주 극소수의 값들이 3천대, 그리고 최대로 5천이 약간 넘는 값이 있음을 확인 가능합니다. 한번 해당 값들을 살펴보겠습니다.

In [ ]:
train[train['Distance']>3000].head(50)

살펴보니 텍사스에서 알래스카를 간다든가, 콜로라도에서 하와이를 간다든가 하는 값이네요. 특별히 값 자체로 이상할건 없지 않나 생각을 합니다. 한번 회귀식에 적합해보고, 영향력관찰치로 보이면 제거하고 Distance>3000의 값들에 대해서는 값을 별도로 배정한다든가 하겠습니다.

앞서 edt와 eat로 만든 값들에 대해서, 소요시간 항목을 따로 만들고 해당 값들에 대해서 Distance값들을 추적해 회귀식을 적합해보도록 하겠읍니다.

In [ ]:
i = 0
while i < 999999:
    if train.loc[i, 'edt'] > 0 and train.loc[i, 'eat'] > 0 :
        train.loc[i, 'time'] = train.loc[i, 'eat'] - train.loc[i, 'edt']
    i += 1
    

이제 값이 생성되었으니 해당 값으로 scatter를 그려서 확인해보겠습니다.

In [ ]:
plt.scatter( train[train['time'].notnull()][['Distance', 'time']]['Distance'], train[train['time'].notnull()][['Distance', 'time']]['time'])

엄

왼쪽 위에 저건 뭐지?

아무튼 크게 두 갈래로 갈라지는데, 아마 제 생각엔 타임존을 역행하는지, 타임존 방향으로 나아가는지가 문제가 아닌가 생각이 드네요. 데이콘 규칙상 외부 정보를 사용할수가 없어서 참 메우기 아쉬운 부분이 있습니다. 

왼쪽 위에 것들은 거리는 극단적으로 짧은데, 시간 소요는 극히 긴 것으로 보아서는 아무래도 time zone을 역행할 때 비행기가 시간선보다 먼저 도착한 케이스(그래서 1시간이 안되게 도착한 케이스)라고 보여지는데, 이게 실제로 되는건지 어떤지는 잘 모르겠네요. 미국에 안살아봐서.

In [ ]:
train[train['time']>1400]

보면 거리상 해당 시간이 나올 수가 없는 구조로 되어 있네요. 아무래도 Estimated_Departure_Time과 Estimated_Arrival_Time이 뒤바뀐거 아니면 시차 timezone을 역행해서 간게 문제가 아닌가 생각이 들긴 하네요.

그래도 괜찮습니다. 어차피 상술하였듯 별도 컬럼을 생성할거기 때문에

우선은 train의 origin_state와 destination_state별로 시간을 새로 설정해야겠습니다. 

In [ ]:
distance = train.groupby(['Origin_State', 'Destination_State'])['Distance'].mean().reset_index()

In [ ]:
deptimes = train.groupby(["Origin_State", "Destination_State"])['edt'].mean().reset_index()

In [ ]:
atimes = train.groupby(['Origin_State', 'Destination_State'])['eat'].mean().reset_index()

In [ ]:
distance = distance.merge(deptimes, how='left', on = ['Origin_State', 'Destination_State']).merge(atimes, how='left', on=['Origin_State', 'Destination_State'])

In [ ]:
distance['times'] = distance['eat']-distance['edt']

In [ ]:
list((set(distance[distance['Origin_State']=='California'].index)\
      &set(distance[distance['Destination_State']=='Texas'].index)))[0]

In [ ]:
distance

In [ ]:
train

In [ ]:
distance[distance['Destination_State']==train.loc[5, 'Destination_State']]

# 위 times 컬럼으로 edt, eat 컬럼 결측값 배정

In [ ]:
i = 0
while i <= 999999:
    if train.loc[i, 'edt'] > 0:
        pass
    else : 
        if train.loc[i, 'eat'] > 0:
            if len(list(set(distance[distance['Origin_State']==train.loc[i, 'Origin_State']].index)&set(distance[distance['Destination_State']==train.loc[i, 'Destination_State']].index))) != 0:
                train.loc[i, 'edt'] = train.loc[i, 'eat'] - distance.loc[list(set(distance[distance['Origin_State']==train.loc[i, 'Origin_State']].index)&set(distance[distance['Destination_State']==train.loc[i, 'Destination_State']].index))[0], 'times']
            else:
                pass
    i += 1

In [ ]:
i = 0
while i <= 999999:
    if train.loc[i, 'eat'] > 0:
        pass
    else : 
        if train.loc[i, 'edt'] > 0:
            if len(list(set(distance[distance['Origin_State']==train.loc[i, 'Origin_State']].index)&set(distance[distance['Destination_State']==train.loc[i, 'Destination_State']].index))) != 0:
                train.loc[i, 'eat'] = train.loc[i, 'edt'] + distance.loc[list(set(distance[distance['Origin_State']==train.loc[i, 'Origin_State']].index)&set(distance[distance\
                ['Destination_State']==train.loc[i, 'Destination_State']].index))[0], 'times']
            else:
                pass
    i += 1

얼추 정리가 된것같으니 다시 한번 보겠습니다

In [ ]:
train.isnull().sum()

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression(fit_intercept=False)

model.fit(pd.DataFrame(train[train['time'].notnull()][['Distance', 'time']]['time']), pd.DataFrame(train[train['time'].notnull()][['Distance', 'time']]['Distance']))

print(model.coef_)
print(model.intercept_)

보면 얼추 coef가 5.3? 5.4? 이정도 나오네요. 분당 얼추 5.3~4마일정도(얼추 10km 좀 안되게) 간다는 이야기인데, 비행기의 속도를 생각해보면 이상하지는 않은 것 같습니다. 제가 항공기 기기나 엔지니어링 자체에 엄청 관심이 많지는 않아서, 틀릴지도 모르겠습니다. 하여튼, 얼추 이 값을 토대로 결측을 메워주도록 하겠습니다.

In [ ]:
train

In [ ]:
dpm = 5.38393855

In [ ]:
train['time'] = train['Distance']/dpm

In [ ]:
i = 0
while i <= 999999:
    if train.loc[i, 'edt'] < 0:
        train.loc[i, 'edt'] += 1440
        train.loc[i, 'eat'] += 1440
    i += 1

해결이 됐으니 정수로 바꿔주도록 합시다

In [ ]:
i = 0
while i <= 999999:
    if train.loc[i, 'edt'] > 0:
        train.loc[i, 'eat'] = round(train.loc[i, 'eat'])
        train.loc[i, 'edt'] = round(train.loc[i, 'edt'])
    i += 1

In [ ]:
train.to_csv("C:/vscode/open/final/training.csv")

이후로도 time에는 한 만개정도의 결측값이 있지만, 당장 뭔가를 할 수는 없는 부분이라 생각합니다. 학습시켜서 자동으로 채우라고 할 수도 있지만 제가 blackbox식으로 채워지는걸 그렇게 좋아하지가 않아서, 나중에 다른 값들을 보고 결정해야 할 것 같아요.

그런 의미에서 일단은 주와 시간을 얼추 해결했으니, 남은 결측치인 항공사를 메워볼까 합니다. 

항공사를 채우는 경우, 이미 주어진 값인 공항과 tail_number값을 통해서 채워볼까 합니다. 

공항별로 항공사를 groupby하고, tailnumber별로 항공사를 groupby한 후, 둘의 교집합이 있다면 그 중 하나를, 없다면 공항별 groupby된 것에서 하나를 뽑을까 합니다. 

In [ ]:
oa = train.groupby('Origin_Airport')['Airline'].unique().reset_index().dropna()

In [ ]:
da = train.groupby('Destination_Airport')['Airline'].unique().reset_index().dropna()

In [ ]:
ta = train.groupby('Tail_Number')['Airline'].unique().reset_index().dropna()

In [ ]:
import random

In [ ]:
oairlineairport = train.groupby('Origin_Airport')['Airline'].unique().reset_index()

dairlineairport = train.groupby('Destination_Airport')['Airline'].unique().reset_index()

In [ ]:
carr_id = train.groupby('Carrier_ID(DOT)')['Airline'].unique().reset_index()

In [ ]:
ccod = train.groupby("Carrier_Code(IATA)")['Airline'].unique().reset_index()

In [ ]:
i = 0
while i <= 999999:
    j = 0
    k = 0
    l = 0
    if isinstance(train.loc[i, 'Airline'], float):
        
        while j < len(oairlineairport['Origin_Airport']):
            if train.loc[i, 'Origin_Airport'] == oairlineairport.loc[j, 'Origin_Airport']:
                
                while k < len(dairlineairport['Destination_Airport']):
                    
                    if train.loc[i, 'Destination_Airport'] == dairlineairport.loc[k, 'Destination_Airport']:
                        
                        while l < len(carr_id['Carrier_ID(DOT)']) :
                            if train.loc[i, 'Carrier_ID(DOT)'] == carr_id.loc[l, 'Carrier_ID(DOT)']:
                                
                                train.loc[i, 'Airline'] = list(set(oairlineairport.loc[j, 'Airline'])&set(dairlineairport.loc[k, 'Airline'])&set(carr_id.loc[l, 'Airline']))[random.randint(0, (len(set(oairlineairport.loc[j, 'Airline'])&set(dairlineairport.loc[k, 'Airline'])&set(carr_id.loc[l, 'Airline']))-1))]
                                break
                                
                            l += 1
                        break
                        
                    k += 1
                break
                
            j += 1
    i += 1

In [ ]:
i = 0
while i <= 999999:
    j = 0
    if isinstance(train.loc[i, 'Airline'], float):
        while j < len(oa['Origin_Airport']):
            k = 0
            if oa.loc[j, 'Origin_Airport'] == train.loc[i, 'Origin_Airport']:
                while k < len(da['Destination_Airport']):
                    l = 0
                    if da.loc[k, 'Destination_Airport'] == train.loc[i, 'Destination_Airport']:
                        
                        while l < len(ta['Tail_Number']):
                            if ta.loc[l, 'Tail_Number'] == train.loc[i, 'Tail_Number']:
                                if len(set(oa.loc[j, 'Airline'])&set(da.loc[k, 'Airline'])&set(ta.loc[l, 'Airline'])) == 0:
                                    train.loc[i, 'Airline'] = list(set(oa.loc[j, 'Airline'])&set(da.loc[k, 'Airline']))[random.randint(0, int(len(list(set(oa.loc[j, 'Airline'])&set(da.loc[k, 'Airline']))))-1)]
                                    break
                                else:
                                    train.loc[i, 'Airline'] = list(set(oa.loc[j, 'Airline'])&set(da.loc[k, 'Airline'])&set(ta.loc[l, 'Airline']))[random.randint(0, int(len(list(set(oa.loc[j, 'Airline'])&set(da.loc[k, 'Airline'])&set(ta.loc[l, 'Airline']))))-1)]
                                    break
                                break
                            l += 1
                        break        
                    k += 1
                break        
            j += 1
    i += 1

In [ ]:
i = 0
while i <= 999999:
    j = 0
    if isinstance(train.loc[i, 'Airline'], float):
        while j < len(oa['Origin_Airport']):
            k = 0
            if oa.loc[j, 'Origin_Airport'] == train.loc[i, 'Origin_Airport']:
                while k < len(da['Destination_Airport']):
                    l = 0
                    if da.loc[k, 'Destination_Airport'] == train.loc[i, 'Destination_Airport']:
                        
                        while l < len(ccod['Carrier_Code(IATA)']):
                            if ccod.loc[l, 'Carrier_Code(IATA)'] == train.loc[i, 'Carrier_Code(IATA)']:
                                if len(set(oa.loc[j, 'Airline'])&set(da.loc[k, 'Airline'])&set(ccod.loc[l, 'Airline'])) == 0:
                                    train.loc[i, 'Airline'] = list(set(oa.loc[j, 'Airline'])&set(da.loc[k, 'Airline']))[random.randint(0, int(len(list(set(oa.loc[j, 'Airline'])&set(da.loc[k, 'Airline']))))-1)]
                                    break
                                else:
                                    train.loc[i, 'Airline'] = list(set(oa.loc[j, 'Airline'])&set(da.loc[k, 'Airline'])&set(ccod.loc[l, 'Airline']))[random.randint(0, int(len(list(set(oa.loc[j, 'Airline'])&set(da.loc[k, 'Airline'])&set(ta.loc[l, 'Airline']))))-1)]
                                    break
                                break
                            l += 1
                        break        
                    k += 1
                break        
            j += 1
    i += 1

반복문이 좀 어지럽네요. 약간 파도치는거같기도 하고. 저는 컴퓨터 사양이 별로 안좋아서 이거 돌아가는동안 영화한편 보고 왔습니다.

gpu에 올려놓고 할까 하는 후회를 좀 하긴 했는데, 어차피 돈받고 하는것도 아니니까 더 귀찮아지는게 싫어서 그냥 넘어갔습니다. 일단 토치 패키지는 다운받아놨는데, 주피터로 열려고 하니까 에러가 나더라구요. vscode랑 파이썬 버전이 달라서 그런가.

In [ ]:
train.isnull().sum()

In [ ]:
i = 0
while i <= 999999:
    if isinstance(train.loc[i, 'Airline'], float):
        j = 0
        k = 0
        while j < 52:
            if train.loc[i, 'Origin_Airport'] == oa.loc[j, 'Origin_Airport']:
                while k < 52:
                    if train.loc[i, 'Destination_Airport'] == da.loc[k, 'Destination_Airport']:
                        train.loc[i, 'Airline'] = list(set(oa.loc[j, 'Airline'])&set(da.loc[k, 'Airline']))[random.randint(0, len(list(set(oa.loc[j, 'Airline'])&set(da.loc[k, 'Airline'])))-1)]
                    k += 1
            j += 1    
        
    i += 1

In [ ]:
train.isnull().sum()

다 돌려놓고 생각을 해 보니까 그냥 중첩반복문을 이렇게 몇개나 쓰지 말고 한개 안에 다양한 칼럼들이나 조건들을 욱여넣어서 사용하는게 더 시간을 적게 먹었을텐데 다음에 작성할때는 주의해야겠습니다.

그래도 많이 줄었네요. 썩 만족스럽지는 않지만, 아무튼 됐습니다. Carrier_Code와 Carrier_ID의 결측치를 함게 메우면서 했으면 결과가 조금 더 잘 나왔지 싶기도 하네요. 하여튼 이제 남은 값들을 airport를 통해서 메우겠습니다

In [ ]:
while train['Airline'].isnull().sum() > 50:
    i = 0
    while i <= 999999:
        j = 0
        if isinstance(train.loc[i, 'Airline'], float):
            while j < len(oa['Origin_Airport']):
                if train.loc[i, 'Origin_Airport'] == oa.loc[j, 'Origin_Airport']:
                    train.loc[i, 'Airline'] = oa.loc[j, 'Airline'][random.randint(0, int(len(oa.loc[j, 'Airline'])-1))]
                j += 1
        i += 1

In [ ]:
train.isnull().sum()

남은건 적당히 state 값을 통해서 배치하겠습니다.

In [ ]:
os = train.groupby("Origin_State")['Airline'].unique().reset_index()

In [ ]:
i = 0
while i <= 999999:
    j = 0
    if isinstance(train.loc[i, 'Airline'], float):
        while j < 52:
            if train.loc[i, 'Origin_State'] == os.loc[j, 'Origin_State']:
                train.loc[i, 'Airline'] = os.loc[j, 'Airline'][random.randint(0, len(os.loc[j, 'Airline'])-1)]
            j += 1
    i += 1

In [ ]:
train.isnull().sum()

없어졌네요. 그러면 이제 남은 값은 edt, eat 결측값 처리랑, 아까 그 YNG 공항 처리네요. 먼저 그 공항 처리부터 가볍게 해 보겠습니다.

In [ ]:
missing_airport = train[train['Destination_State'].isnull()]['Airline']

In [ ]:

train[train['Airline']=='Allegiant Air']['Destination_State'].mode()
    

플로리다가 모드값이네요. 아까 오하이오였나 그랬던거같은데, 틀리긴 했는데 그래도 해당 값으로 배정하겠습니다.

In [ ]:
train['Destination_State'] = train['Destination_State'].fillna('Florida')

In [ ]:
train.isnull().sum()

In [ ]:
train = train.drop(columns = ['Carrier_Code(IATA)', 'Carrier_ID(DOT)'])

In [ ]:
train.describe()

In [ ]:
train.to_csv("C:/vscode/open/final/training2.csv")

In [ ]:
train = pd.read_csv("C:/vscode/open/final/training2a.csv")

# edt, eat 칼럼 정리, 날짜변환

이제 1440(24 * 60)을 는 & 음수인 edt값을 정리하고, 그걸 날짜에 추가해주도록 하겠습니다.

In [ ]:
i = 0
while i <= 999999:
    if train.loc[i, 'eat'] >= 1440:
        train.loc[i, 'eat'] -= 1440
        train.loc[i, 'aday'] += 1
    i += 1

In [ ]:
train['edt'].describe()

In [ ]:
train['aday'].describe()

In [ ]:
i = 0
while i <= 999999:
    if train.loc[i, 'aday'] >= 32:
        train.loc[i, 'amonth'] += 1
        train.loc[i, 'aday'] = 1
    i += 1

2월에 윤달이 있을수는 있지만, 원본 데이터에 아무런 정보가 없기 때문에 그냥 넘겼습니다. 뭐 그런 케이스가 몇개나 되겠어요.

In [ ]:
i = 0
while i <= 999999:
    if train.loc[i, 'amonth'] == 2:
        if train.loc[i, 'aday'] >= 29:
            train.loc[i, 'amonth'] += 1
            train.loc[i, 'aday'] -= 28
    elif train.loc[i, 'amonth'] in [4, 6, 9, 11]:
        if train.loc[i, 'aday'] >= 31:
            train.loc[i, 'amonth'] += 1
            train.loc[i, 'aday'] -= 30
    i += 1

In [ ]:
train.isnull().sum()

# 테이블 컬럼 정리

이제 안 쓰는 칼럼들, 중간중간 백업-다운로드를 반복하면서 생긴 컬럼들이나 인덱스들을 정리하겠습니다. index=1000000 행은 왜 생긴거지? 아무튼 그렇게 됐습니다.

In [ ]:
train.drop(columns = ['Month', 'Day_of_Month'], inplace=True)

In [ ]:
train.drop(columns = 'Unnamed: 0', inplace=True)

In [ ]:
train.drop(columns = ['Estimated_Departure_Time', 'Estimated_Arrival_Time'], inplace=True)

In [ ]:
train = train.drop(index = 1000000)

In [ ]:
train

In [ ]:
train = train[['edt', 'eat', 'dmonth', 'dday', 'amonth', 'aday',
       'Origin_Airport', 'Origin_State', 'Destination_Airport',
       'Destination_State', 'Distance', 'Airline', 'Tail_Number', 'time', 'Delay']]

In [ ]:
train.isnull().sum()

# 모델 적합

여기서 모델 적합은 Delay가 null이 아닌 것만 사용하겠습니다.

In [ ]:
train.to_csv("C:/vscode/open/final/training3a.csv")

In [ ]:
train = pd.read_csv("C:/vscode/open/final/training3a.csv")

In [ ]:
data = train[train['Delay'].notnull()]

In [ ]:
target = data['Delay']
data.drop(columns = 'Delay', inplace=True)

In [ ]:
data

In [ ]:
target = target.replace('Not_Delayed', 0).replace('Delayed', 1)

그리고 아까 까먹었는데, edt랑 eat를 다시 시간단위로 풀겠습니다.
원래처럼 7:20 하는 식으로 바꾼 다음, 분단위를 시간단위 소수점으로 바꿔서 적합할까 합니다.

In [ ]:
data = data.reset_index()

In [ ]:
data

In [ ]:
target=target.reset_index()

In [ ]:
i = 0
while i < len(data['edt']):
    data.loc[i, 'edt'] = float(data.loc[i, 'edt']//60)*100 + float((data.loc[i, 'edt']%60)/60)
    data.loc[i, 'eat'] = float(data.loc[i, 'eat']//60)*100 + float((data.loc[i, 'eat']%60)/60)
    i += 1

이거는 약간 논외의 이야기인데, 사실 달-월을 연 단위로 표준화해서 1-364 스케일링을 할까도 생각했었고, 실제로 작업도 했었는데, 영 결과가 별로더라구요. 그렇다고 그냥 date타입으로 쓰기에는 연도가 항상 기입돼서 나와갖고 그게 좀 불편해서 그냥 저렇게 사용하게 되었습니다.

이제 Airport, State, Airline, Tail_Number를 LabelEncoding하면 끝나겠네요. 마음같아서는 one-hot encoding을 통해서 더 다양한 모델을 적용해보고 싶었지만 램도 모자라고(이게 굉장히 중요) 컴퓨터 성능도 마뜩치가 않아서 일단은 라벨인코딩으로 진행해야 할 듯 하네요. 아쉽습니다.

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [ ]:
train.to_csv("C:/vscode/open/final/training3b.csv")

In [ ]:
encoder1 = encoder.fit(data['Origin_Airport'])
data["Origin_Airport"] = encoder1.transform(data['Origin_Airport'])
data['Destination_Airport'] = encoder1.transform(data['Destination_Airport'])

In [ ]:
encoder2 = encoder.fit(data['Destination_State'])
data['Origin_State'] = encoder2.transform(data['Origin_State'])
data['Destination_State'] = encoder2.transform(data['Destination_State'])

In [ ]:
encoder3 = encoder.fit(data['Airline'])
data['Airline'] = encoder3.transform(data['Airline'])

In [ ]:
data2 = pd.read_csv("C:/vscode/open/final/dta.csv")

In [ ]:
encoder4 = encoder.fit(data2['Tail_Number'])
data2['Tail_Number'] = encoder4.transform(data2['Tail_Number'])

In [ ]:
encoder4 = encoder.fit(data['Tail_Number'])
data['Tail_Number'] = encoder4.transform(data['Tail_Number'])

In [ ]:
data = data.drop(columns = 'index')

In [ ]:
data = data.drop(columns = 'Unnamed: 0')

이제 데이터 dtype을 확인하고, 필요한 경우가 생긴다면 조정해주도록 하겠습니다.

In [ ]:
data.dtypes

In [ ]:
if 'Unnamed: 0.1' in data.columns:
    data.drop(columns = 'Unnamed: 0.1', inplace=True)

변수들 중, Airport, State, Airline, Tail_Number는 숫자로 남겨두는것보다 카테고리컬로 남겨두는게 더 낫다고 생각하기 때문에, 해당 칼럼들을 카테고리(unordered)로 변환하겠습니다.

In [ ]:
data['Origin_Airport']=data['Origin_Airport'].astype('category')

In [ ]:
data['Destination_Airport']=data['Destination_Airport'].astype('category')

In [ ]:
data['Origin_State'] = data['Origin_State'].astype('category')

In [ ]:
data['Destination_State'] = data['Destination_State'].astype('category')

In [ ]:
data['Airline'] = data['Airline'].astype('category')

In [ ]:
data['Tail_Number'] = data['Tail_Number'].astype('category')

In [ ]:
col = ['Origin_Airport', 'Destination_Airport', 'Origin_State', 'Destination_State', 'Airline', 'Tail_Number']

In [ ]:
for i in col:
    data[i] = data[i].cat.as_unordered()

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

In [ ]:
scaler.fit(pd.DataFrame(data['Distance']))

In [ ]:
data.dtypes

그냥 처음부터 함수로 할걸 그랬네요.

하여튼 이젠 대충 다 됐으니, 모델 적합을 시켜보겠습니다. 

In [ ]:
target

In [ ]:
target.drop(columns = 'index', inplace=True)

In [ ]:
target['Delay'] = target['Delay'].astype('int')

In [ ]:
i = 0
while i < len(target['Delay']):
    if target.loc[i, 'Delay'] == 0:
        target.loc[i, 'Delayed'] = 0
        target.loc[i, 'Not_Delayed'] = 1
    else:
        target.loc[i, 'Delayed'] = 1
        target.loc[i, 'Not_Delayed'] = 0
    i += 1

In [ ]:
target.drop(columns = 'Delay', inplace=True)

target['Delayed'] = target['Delayed'].astype('int')
target['Not_Delayed'] = target['Not_Delayed'].astype('int')

In [ ]:
from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(data, target, test_size=0.25)

저는 왠지는 몰라도 0.25가 좀 손이 가더라구요. 뭔가 비율이 안정돼보여서 그런가. 

모델은 xgboost xgbclassifier을 사용해볼까 합니다. 이것도 영 별로면 랜덤포레스트나 lightgbm을 사용할수는 있을텐데, lightgbm은 딱히 익숙하지가 않아서 그냥 xgb로 잘 나왔으면 좋겠네요.

In [ ]:
import xgboost as xgb

In [ ]:
from xgboost import XGBClassifier

In [ ]:
dtrain = xgb.DMatrix(data = xtrain, label = ytrain, enable_categorical=True)

dtest = xgb.DMatrix(data = xtest, label = ytest, enable_categorical=True)

params = {'max_depth':14, 
          'learning_rate':0.03,  
          'scale_pos_weight':5, 
          'min_child_weight':3, 
          'objective' : 'binary:logistic',
          'eval_metric':'logloss',
          'booster':'gbtree'}

num_rounds = 150

In [ ]:
xgbmodel = xgb.train(params = params, num_boost_round = num_rounds, dtrain=dtrain)

In [ ]:
pred = xgbmodel.predict(dtest)

규칙상 모델 평가는 log_loss를 통해 이루어져서, 저도 평가 모델로 로그로스를 채용하도록 하겠습니다.

In [ ]:
from sklearn.metrics import log_loss

In [ ]:
log_loss(ytest, pred)

In [ ]:
pred

0.47정도면 썩 엄청 좋지는 않은데, 그렇다고 엄청 나쁘지도 않긴 하네요. 아마 test파일을 직접 넣으면 이것보다 못하게 나올것같긴 한데, 뭐 됐습니다. 일단 해보면 알겠죠.

이제 위에 있는 전체를 test에도 동일하게 돌려서 만들어보면 되지 않을까 합니다. 약간 벌써부터 컴퓨터 하루종일 돌아갈거 생각하면 약간 아찔하긴 한데, 해봐야겠죠.

In [ ]:
testdata = pd.read_csv("C:/vscode/open/final/testdata4.csv")

In [ ]:
testdata2 = pd.read_csv("C:/vscode/open/final/testdata4.csv")

테스트데이터는 동일한 프로세스에서 인코딩을 앞둔 지점에서 끌고왔습니다. 뭐 거의 다했죠. 이번에도 영화 한편 보고 왔습니다.

어쩌면 유튜브... 넷플릭스... 라프텔... 현대인의 필수 서비스가 아닐지...

이하의 encoding들은 제가 한번 까먹어서 차후에 덧붙여진 부분입니다.

encoding을 까먹고 했을 때 점수가 더 잘 나오던데, 도대체 어떤 매커니즘으로 정답지를 만든건지 잘 모르겠네요.

TestData 인코딩 및 결측치 처리

In [ ]:
for label in np.unique(testdata['Origin_Airport']):
    if label not in encoder1.classes_:
        encoder1.classes_ = np.append(encoder1.classes_, label)

In [ ]:
testdata['Origin_Airport'] = encoder1.transform(testdata['Origin_Airport'])
testdata['Destination_Airport'] = encoder1.transform(testdata['Destination_Airport'])

In [ ]:
testdata['Origin_State'].mode()

In [ ]:
testdata['Destination_State'].mode()

In [ ]:
testdata['Origin_State'] = testdata['Origin_State'].fillna('California')

In [ ]:
testdata['Destination_State'] = testdata['Destination_State'].fillna('California')

In [ ]:
testdata['Origin_State'] = encoder2.transform(testdata['Origin_State'])
testdata['Destination_State'] = encoder2.transform(testdata['Destination_State'])

In [ ]:
testdata['Airline'].mode()

In [ ]:
testdata['Airline'] = testdata['Airline'].fillna("Southwest Airlines Co.")

In [ ]:
for label in np.unique(testdata['Airline']):
    if label not in encoder3.classes_:
        encoder3.classes = np.append(encoder3.classes_, label)

In [ ]:
testdata['Airline'] = encoder3.transform(testdata['Airline'])

In [ ]:
for label in np.unique(testdata['Tail_Number']):
    if label not in encoder4.classes_:
        encoder4.classes_ = np.append(encoder4.classes_, label)

In [ ]:
testdata['Tail_Number'] = encoder4.transform(testdata['Tail_Number'])

In [ ]:
samsub = pd.read_csv("C:/vscode/open/sample_submission.csv")

In [ ]:
testdata.isnull().sum()

In [ ]:
testdata['Distance'] = scaler.transform(pd.DataFrame(testdata['Distance']))

In [ ]:
testdata

In [ ]:
data

아직 State와 Airline에 약간 결측값이 남아있네요. 이번 데이터에는 YNG같은 공항이 몇개 더 있나봅니다. 어쩔 수 없죠. 수가 많지는 않으니 그냥 채워주겠습니다.

In [ ]:
testdata.drop(columns = ['Unnamed: 0'], inplace=True)

이제 필요한 컬럼들의 데이터타입을 변경하도록 하겠습니다. 위에서 한 대로 state, airport, airline, tailnumber 값들을 변경하겠습니다.

In [ ]:
clist = ['Origin_Airport', 'Origin_State', 'Destination_State', 'Destination_Airport', 'Airline', 'Tail_Number']

In [ ]:
for i in clist:
    testdata[i] = testdata[i].astype('category')
    testdata[i] = testdata[i].cat.as_unordered()

In [ ]:
testdata

In [ ]:
testdata.dtypes

정리가 다 됐네요. 그럼 샘플 서브미션 파일을 정리하고 모델에 적합하겠습니다.

# Model Predict I

In [ ]:
samid = samsub['ID']

In [ ]:
samsub.drop(columns = 'ID', inplace=True)

In [ ]:
samsub = samsub[['Delayed', 'Not_Delayed']]

In [ ]:
dsubmit = xgb.DMatrix(data = testdata, label = samsub, enable_categorical=True)

In [ ]:
submis1 = xgbmodel.predict(dsubmit)

In [ ]:
submis1 = pd.DataFrame(submis1)

In [ ]:
samsub1 = pd.concat([samid, submis1], axis=1)

In [ ]:
samsub1.columns = ['ID', 'Delayed', 'Not_Delayed']

In [ ]:
samsub1 = samsub1[["ID", "Not_Delayed", 'Delayed']]

In [ ]:
samsub1

In [ ]:
samsub1 = samsub1.set_index("ID")

In [ ]:
samsub1

In [ ]:
samsub1.to_csv("C:/vscode/open/submission/samsub9.csv")

# Model Predict II

얻어낸 모델의 확률(XGBOOST XGBClassifier의 Binary Logistic은 이진변수의 각 항목에 대한 확률값을 리턴합니다)을 합 1로 맞춰 주겠습니다.

In [ ]:
samsub2 = samsub1.copy()

In [ ]:
samsub2 = samsub2.reset_index()

In [ ]:
i = 0
while i < 1000000:
    samsub2.loc[i, 'Not_Delayed'] = samsub2.loc[i, 'Not_Delayed']/(samsub2.loc[i, 'Not_Delayed']+samsub2.loc[i, "Delayed"])
    samsub2.loc[i, "Delayed"] = samsub2.loc[i, 'Delayed']/(samsub2.loc[i, 'Not_Delayed']+samsub2.loc[i, "Delayed"])
    i += 1

In [ ]:
samsub2 = samsub2.set_index("ID")

In [ ]:
samsub2.to_csv("C:/vscode/open/submission/samsub10.csv")

dacon 사이트가 난리가 났길래(train된 모델의 delay 비율보다 공개되지 않은 정답의 delay 비율이 훨씬 높은것이 아니냐 하는 문제가 터졌길래) 저도 한번 확률을 임의로 주작해서 띄워보기로 했습니다. 

In [ ]:
samsub3 = samsub2.copy().reset_index()

In [ ]:
i = 0
while i <= 999999:
    samsub3.loc[i, 'Not_Delayed'] = samsub3.loc[i, 'Not_Delayed'] - 0.1
    samsub3.loc[i, 'Delayed'] = samsub3.loc[i, 'Delayed'] +0.1
    i += 1

In [ ]:
samsub3=pd.concat([samid, samsub3], axis=1)

In [ ]:
samsub3 = samsub3.set_index("ID")

In [ ]:
samsub3

지금보니까 인코딩도 안하고 그냥 학습시켜서 제출했는데 지금까지 제출한 다른 모든 파일보다 더 높은 점수를 받았네요

안그래도 그것때문에 사이트에 난리가 났던데 도대체 정답 테이블에 어떤 알고리즘을 넣은건지... 갑자기 좀 빡치긴 하는데 일단 계속 하겠습니다

결과 : sub3값이 제일 결과가 높네요. 인코딩을 하든, 하지 않았든, 유의미하게 결과물을 임의로 바꾼 것의 평가점수가 더 좋게 나왔습니다. 이건 뭐 제가 작업한게 문제인건지...